In [ ]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, LayerNormalization, Activation, ZeroPadding2D, AveragePooling2D, GaussianNoise
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, ReLU
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.models import load_model, save_model
from keras.datasets.cifar10 import load_data
from skimage.transform import resize
from keras import initializers, backend
from tensorflow.keras.utils import to_categorical
import time
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from progressbar import progressbar
import cv2
from Augmentor import Pipeline

In [ ]:
training_data = np.load('/content/drive/MyDrive/Art/CPGAN128x128.npy')
form = np.load('/content/drive/MyDrive/Art/Form.npy')

In [ ]:
training_data.shape

(18312, 128, 128, 3)

In [ ]:
random_dim = 100
PREVIEW_ROWS = 6
PREVIEW_COLS = 6
PREVIEW_MARGIN = 0
SAVE_FREQ = 10
IMAGE_SIZE = 128
WIDTH = 128
HEIGHT = 128
CONTROL_SIZE_SQRT = 6
CHANNELS = 3
EPOCHS =  50000
START_DIM = 4

FOLDER_PATH = "/content/drive/MyDrive/PhotoComposition/CPGAN-Images/"

In [ ]:
labels = []
files = []
label = 0
for folder in tqdm(os.listdir(FOLDER_PATH)):
  path = os.path.join(FOLDER_PATH, folder)
  for filename in os.listdir(path):
    labels.append(label)
    files.append(filename)
  label += 1


  0%|          | 0/8 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def save_images(cnt, noise, generator):
  image_array = np.full((PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)), PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3), 255, dtype=np.uint8)
  generated_images = generator.predict(noise, verbose = 0)
  generated_images = 0.5 * generated_images + 0.5
  image_count = 0
  for row in range(PREVIEW_ROWS):
    for col in range(PREVIEW_COLS):
      r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
      c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
      image_array[r:r + IMAGE_SIZE, c:c + IMAGE_SIZE] = generated_images[image_count] * 255
      image_count += 1
  output_path = '/content/drive/MyDrive/Art/Outputs/output107'
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  filename = os.path.join(output_path, f"trained-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [ ]:
def get_optimizer():
  return RMSprop(lr = .0001, clipvalue = 1.0, decay = 1e-8)
def get_generator():
  gen_input = Input(shape=random_dim)
  generator = Sequential()

  generator.add(Dense(512 * START_DIM * START_DIM, input_dim=random_dim))
  generator.add(LeakyReLU(alpha=0.2))
  generator.add(Reshape((START_DIM, START_DIM, 512)))

  generator.add(UpSampling2D());
  generator.add(Conv2DTranspose(1024, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(UpSampling2D());
  generator.add(Conv2DTranspose(512, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(UpSampling2D());
  generator.add(Conv2DTranspose(256, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(UpSampling2D());
  generator.add(Conv2DTranspose(128, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(UpSampling2D());
  generator.add(Conv2DTranspose(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(Conv2D(CHANNELS, (4, 4), activation='tanh', padding='same', kernel_initializer='he_normal'))

  generated_image = generator(gen_input)

  generator.summary()

  return Model(gen_input, generated_image)

def get_discriminator():
  models = []
  disc_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
  discriminator = Sequential()
  # discriminatorClass = Sequential()

  discriminator.add(disc_input)

  discriminator.add(Conv2D(64, (1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(AveragePooling2D());
  discriminator.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(AveragePooling2D());
  discriminator.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(AveragePooling2D());
  discriminator.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(AveragePooling2D());
  discriminator.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(AveragePooling2D());
  discriminator.add(Conv2D(1024, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_normal'))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(Flatten())
  # discriminatorClass.add(discriminator)

  discriminator.add(Dense(1, activation='sigmoid'))

  # discriminatorClass.add(Dense(1024))
  # discriminatorClass.add(LeakyReLU(0.2))

  # discriminatorClass.add(Dense(512))
  # discriminatorClass.add(LeakyReLU(0.2))

  # discriminatorClass.add(Dense(8, activation='softmax'))

  discriminator.summary()

  models.append(discriminator(disc_input))
  # models.append(discriminatorClass(disc_input))


  discriminator = Model(disc_input, models)

  optimizer = RMSprop(
       lr = .01,
       clipvalue = 1.0,
       decay = .01/
   )
  # optimizer = Adam(lr=0.0002, beta_1=0.9)
  discriminator.compile(loss=['binary_crossentropy'], optimizer=optimizer)

  return discriminator

def get_batch_labels(idx):
  ID = []
  for i in idx:
    ID.append(labels[i])
  return np.array(ID)

def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss=['binary_crossentropy'], optimizer=optimizer)

  discriminator.trainable = True
  return gan

X_train = training_data
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 100))

def train(epochs=1, batchSize=128):
  batchCount = X_train.shape[0] / batchSize
  print(X_train.shape[0])
  print('Epochs:', epochs)
  print('Batch size:', batchSize)
  print('Batches per epoch:', batchCount)

  adam = get_optimizer()
  generator = get_generator()
  # gen_load = load_model('/content/drive/MyDrive/Art/SavedModels/GAN_OUT105-25000-GEN.h5')
  discriminator = get_discriminator()
  # disc_load = load_model('/content/drive/MyDrive/Art/SavedModels/GAN_OUT105-25000-DISC.h5')

  gan = get_gan_network(discriminator, random_dim, generator, adam)

  y_real = np.ones((batchSize, 1))
  y_fake = np.zeros((batchSize, 1))

  for step in tqdm(range(EPOCHS)):
    latent_vectors = np.random.normal(size=(batchSize, random_dim))
    generated = generator.predict(latent_vectors, verbose = 0)

    idx = np.random.randint(0, training_data.shape[0], batchSize)
    real = training_data
    # real[batchSize - 1] = form[0]

    d_loss = discriminator.train_on_batch(real, np.ones((batchSize, 1)) )
    d_loss = discriminator.train_on_batch(generated, np.zeros((batchSize, 1)) )

    latent_vectors = np.random.normal(size=(batchSize, random_dim))
    misleading_targets = np.ones((batchSize, 1))

    a_loss = gan.train_on_batch(latent_vectors, misleading_targets)

    if step % 250 == 0:
      save_images(step, fixed_noise, generator)

    if step % 5000 == 0 and step != 0:
      discriminator.trainable = False
      gan.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-GAN.h5")
      discriminator.trainable = True
      generator.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-GEN.h5")
      discriminator.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-DISC.h5")

  discriminator.trainable = False
  gan.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-GAN.h5")
  discriminator.trainable = True
  generator.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-GEN.h5")
  discriminator.save(f"/content/drive/MyDrive/Art/SavedModels/GAN_OUT107-{step}-DISC.h5")
  print("Saved Model!")



if __name__ == '__main__':
     train(5000, 32)

18312
Epochs: 5000
Batch size: 32
Batches per epoch: 572.25


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8192)              827392    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 8192)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 512)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 8, 8, 512)        0         
 )                                                               
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 1024)       4719616   
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 8, 8, 1024)        0

  0%|          | 0/50000 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-5d2e4df40699>", line 181, in <module>
    train(5000, 32)
  File "<ipython-input-6-5d2e4df40699>", line 159, in train
    a_loss = gan.train_on_batch(latent_vectors, misleading_targets)
  File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 2146, in train_on_batch
    logs = tf_utils.sync_to_numpy_or_python_type(logs)
  File "/usr/local/lib/python3.8/dist-packages/keras/utils/tf_utils.py", line 607, in sync_to_numpy_or_python_type
    return tf.nest.map_structure(_to_single_numpy_or_python_type, tensors)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/nest.py", line 916, in map_structure
    structure[0], [func(*x) for x in entries],
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/nest.py", line 916, in 

KeyboardInterrupt: ignored